In [53]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [54]:
fp = 'data/observations_train.csv'
info = 'data/series.csv'
df = pd.read_csv(fp)
info_df = pd.read_csv(info)
df.head()

,series_id,date,value
0,AAA10Y,2000-01-03 00:00:00.0000000,1.17
1,AAA10Y,2000-01-04 00:00:00.0000000,1.20
2,AAA10Y,2000-01-05 00:00:00.0000000,1.16
3,AAA10Y,2000-01-06 00:00:00.0000000,1.15
4,AAA10Y,2000-01-07 00:00:00.0000000,1.17


In [55]:
len(df['series_id'].unique())

68

In [56]:
df.isnull().sum() / len(df) #2.6% of our data are missing

series_id    0.000000
date         0.000000
value        0.025972
dtype: float64

In [57]:
df[df['value'].isnull()].index[:10] # Missing values are pretty spread out

Int64Index([10, 35, 79, 105, 131, 175, 200, 233, 255, 260], dtype='int64')

In [58]:
df['date'] = pd.to_datetime(df['date'])
df['date']

0        2000-01-03
1        2000-01-04
2        2000-01-05
3        2000-01-06
4        2000-01-07
            ...    
103299   2017-12-27
103300   2017-12-28
103301   2017-12-29
103302   2017-12-30
103303   2017-12-31
Name: date, Length: 103304, dtype: datetime64[ns]

In [59]:
def extract_frequency(freq):
    daily_tickers = info_df[info_df['frequency'].str.contains(freq)]['series_id']
    return df[df['series_id'].isin(daily_tickers)]

In [60]:
from tqdm import tqdm

#Function to compute additional derived features
#It iterates through each existing feature and compute key statistics (rolling) for multiple lookback periods.
def derive_features(df):
    features = df.columns
    #Iterate through each column in df
    for col in tqdm(features):
        if col[1] != 'SP500':
            #Daily change
            df[col[1]+'_ret'] = df[col].pct_change()
            df[col[1]+'_diff'] = df[col].diff()
            #Lookback periods
            lookback = [5,21, 63, 126, 252]#1 week, 1 month, 1 quarter, half year, 1 year
            
            #Compute key statistics for each lookback period
            for i in lookback:
                df[col[1]+'_ret_' + str(i)] = df[col]/df[col].shift(i)-1
                #Return-based
                df[col[1]+'_rolling_mean_' + str(i)] = df[col[1]+'_ret'].rolling(i).mean()
                df[col[1]+'_rolling_median_' + str(i)] = df[col[1]+'_ret'].rolling(i).median()
                df[col[1]+'_rolling_stdev_' + str(i)] = df[col[1]+'_ret'].rolling(i).std()
                df[col[1]+ '_rolling_skew_' + str(i)] = df[col[1]+'_ret'].rolling(i).skew()
                df[col[1]+ '_rolling_kurt_' + str(i)] = df[col[1]+'_ret'].rolling(i).kurt()
                df[col[1]+ '_rolling_min_' + str(i)] = df[col[1]+'_ret'].rolling(i).min()
                df[col[1]+ '_rolling_max_' + str(i)] = df[col[1]+'_ret'].rolling(i).max()
                df[col[1]+ '_rolling_corr_' + str(i)] = df[col[1]+'_ret'].rolling(i).corr()
                df[col[1]+ '_ewma_' + str(i)] = pd.DataFrame.ewm(df[col], span= i).mean()
                #Relative level
                df[col[1]+'_rel_lvl_mean_' + str(i)] = df[col]/df[col].rolling(i).mean()
                df[col[1]+'_rel_lvl_min_' + str(i)] = df[col]/df[col].rolling(i).min()
                df[col[1]+'_rel_lvl_max_' + str(i)] = df[col]/df[col].rolling(i).max()                        
    return df

In [61]:
# preprocessed = df.pivot_table(columns='series_id', index='date').ffill()
# preprocessed.to_csv('data/preprocessed_orig_features.csv')
one_year = pd.to_timedelta('1y')

y = df[df['series_id'] == 'SP500']
y = y.set_index('date')
y = y.drop('series_id', axis=1)
y = y['value'].interpolate('time')
min_cutoff = y.index.min() - one_year

X = df[df['series_id'] != 'SP500']
X = X.set_index('date')
X = X[X.index >= min_cutoff]
X = X.pivot_table(index=X.index, columns='series_id').value
X = X.interpolate('time')

drop_cols = X.isnull().mean(0).sort_values(ascending=False).index[:3]
X = X.drop(drop_cols, axis=1)

X = X.bfill()
result = pd.concat([X, y], axis=1)
result.to_csv('data/preprocessed_filled.csv')

In [62]:
plt.rcParams.update({'figure.figsize':(10,4), 'figure.dpi':120})

# From EDA
seasonal_features = ['BUSAPPWNSAUS', 'CBUSAPPWNSAUS', 'INFECTDISEMVTRACKD', 'T10YIE', 'WLEMUINDXD']

def poly_fit_remove_seasonality(series_id):
    series = result[series_id]
    X = [i % 365 for i in range(0, len(series))]
    y = series.values
    
    degree = 4
    coef = np.polyfit(X, y, degree)
    print('Coefficients: %s' % coef)
    
    curve = list()
    for i in range(len(X)):
        value = coef[-1]
        for d in range(degree):
            value += X[i] ** (degree - d) * coef[d]
        curve.append(value)
    
    diff = list()
    
    for i in range(len(y)):
        value = y[i] - curve[i]
        diff.append(value + series.mean())
    
    diff_srs = pd.Series(diff, index=series.index)
    _, ax = plt.subplots(1, 2, sharey=True)
    ax[0].plot(series, color='indianred')
    ax[1].plot(diff_srs, color='royalblue')
    plt.suptitle(f'{series_id} - Not Seasonally Adjusted vs. Seasonally Adjusted')
    plt.axis('on')
    plt.subplots_adjust(wspace=0.05, hspace=0)
    plt.show()
    return diff

# for f in seasonal_features:
#     result[f'{f}_no_seasonality'] = poly_fit_remove_seasonality(f)

In [63]:
trend_features = ['CUUR0000SA0R',
 'DEXCHUS',
 'DLTIIT',
 'EECTOT',
 'EFFRVOL',
 'MORTGAGE15US',
 'MORTGAGE30US',
 'OBFR',
 'PCUADLVWRADLVWR',
 'PCUARETTRARETTR',
 'PCUASHCASHC',
 'PCUASTDSVASTDSV',
 'PCUATRADEATRADE',
 'PCUATRNWRATRNWR',
 'PCUATTDSVATTDSV',
 'PCUAWHLTRAWHLTR',
 'PCUOMFGOMFG',
 'SP500',
 'TLAACBW027NBOG',
 'TLBACBW027NBOG']

In [64]:
from sklearn.linear_model import LinearRegression
plt.rcParams.update({'figure.figsize':(8,3), 'figure.dpi':120})

sns.set_style('whitegrid')
sns.set_context('paper')
def detrend(series_id):
    srs = result[series_id]
    X = [i for i in range(0, len(srs))]
    X = np.reshape(X, (len(X), 1))
    y = srs.values
    model = LinearRegression()
    model.fit(X, y)
    _, ax = plt.subplots(1, 2, sharey=True)
    trend = model.predict(X)
    ax[0].plot(y, color='royalblue')
    ax[0].plot(trend, color='indianred')
    ax[0].set_title(f'{series_id} - Trend', fontsize=12)
    ax[0].set_ylabel('Value')
    ax[0].set_xlabel('Time')
    detrended = [y[i]-trend[i]+np.mean(srs) for i in range(0, len(srs))]
    ax[1].plot(detrended, color='royalblue')
    ax[1].set_title(f'{series_id} - Detrended', fontsize=12)
    ax[1].set_ylabel('Value')
    ax[1].set_xlabel('Time')
    plt.show()
    return srs
    
# for f in trend_features:
#     try:
#         detrend(f)
#         result[f'{f}_detrend'] = detrend(f)
#     except KeyError:
#         continue

In [65]:
result.to_csv('data/preprocessed_decomposed.csv')

In [98]:
test_fp = 'data/observations_test.csv'
test = pd.read_csv(test_fp)

test['date'] = pd.to_datetime(test['date'])
test.set_index('date', inplace=True)
test_prep = test.pivot_table(columns='series_id', index=test.index).value

one_year = pd.to_timedelta('1y')
test_prep.interpolate('time')

# drop_cols = X.isnull().mean(0).sort_values(ascending=False).index[:3]
# X = X.drop(drop_cols, axis=1)

test_prep_fileld = test_prep.bfill().ffill()
test_prep_fileld
# result = pd.concat([X, y], axis=1)
# # result.to_csv('data/preprocessed_filled.csv')
# result

series_id,AAA10Y,ASEANTOT,BAA10Y,BUSAPPWNSAUS,BUSAPPWNSAUSYY,CBUSAPPWNSAUS,CBUSAPPWNSAUSYY,CUUR0000SA0R,DEXCHUS,DEXUSEU,...,SFTPAGRM158SFRBSF,SOFR,SOFRVOL,T10YIE,TEDRATE,TLAACBW027NBOG,TLBACBW027NBOG,TSIFRGHT,UNRATE,WLEMUINDXD
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,1.06,98.3,1.78,76910.0,26.21,14440.0,22.48,40.2,6.4910,1.2050,...,0.058482,1.83,825.0,2.00,0.28,16746.697,14922.905,133.6,4.1,9.12
2018-01-03,1.06,98.3,1.77,76910.0,26.21,14440.0,22.48,40.2,6.5010,1.2030,...,0.058482,1.83,825.0,1.98,0.31,16746.697,14922.905,133.6,4.1,12.52
2018-01-04,1.01,98.3,1.75,76910.0,26.21,14440.0,22.48,40.2,6.4915,1.2064,...,0.058482,1.83,825.0,2.01,0.31,16783.427,14961.993,133.6,4.1,70.37
2018-01-05,1.03,98.3,1.76,76910.0,26.21,14440.0,22.48,40.2,6.4875,1.2039,...,0.058482,1.83,825.0,2.01,0.33,16783.427,14961.993,133.6,4.1,34.79
2018-01-06,1.01,98.3,1.74,76910.0,26.21,14440.0,22.48,40.2,6.4970,1.1973,...,0.058482,1.83,825.0,2.02,0.28,16783.427,14961.993,133.6,4.1,20.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-08,1.42,97.7,2.14,59170.0,41.05,8100.0,13.76,39.0,6.4664,1.2035,...,-0.010437,0.02,920.0,2.22,0.15,20631.702,18657.164,136.8,14.7,144.50
2021-02-09,1.42,97.7,2.15,59170.0,41.05,8100.0,13.76,39.0,6.4664,1.2035,...,-0.010437,0.05,901.0,2.22,0.15,20631.702,18657.164,136.8,14.7,144.50
2021-02-10,1.43,97.7,2.16,59170.0,41.05,8100.0,13.76,39.0,6.4664,1.2035,...,-0.010437,0.06,884.0,2.21,0.15,20631.702,18657.164,136.8,14.7,144.50
